# Récupération et retraitement des données

## A) Récupération des données

Le projet repose sur deux sources de données principales:
1. Les données relatives aux diagnostics de performance énergétique (DPE) des logements. Source: Ademe.
2. Les données socio-démographiques caractéristiques des communes (dispositif Filosofi). Source: Insee.
Le premier jeu de données est importé via l'API mise à disposition par l'Ademe sur son portail Opendata. Le second jeu de données est lui importé via le package Pyinsee (permettant de disposer d'une version "propre" d'un grand nombre de fichiers de données de l'Insee).

### 1) Récupération du premier jeu de données via l'API publique "APE logements" (portail Open data de l'ADEME)

In [9]:
# Importations:
import requests
import pandas as pd
import geopandas as gpd

#### a) Récupération préalable de l'ensemble des codes communes situés dans la région Ile-de-france:

In [10]:
# Recherche de la Liste des codes de communes

co_com = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6051727/commune_2022.csv")
print(co_com.head())

# Sélection des communes de la région ile-de-France
CODES = co_com.loc[co_com["REG"]==11,"COM"].unique()
co_com2 = co_com.loc[co_com["REG"]==11,]
print(len(co_com2))
print(len(CODES))  #1288

  TYPECOM    COM   REG DEP CTCD  ARR  TNCC                    NCC  \
0     COM  01001  84.0  01  01D  012     5  ABERGEMENT CLEMENCIAT   
1     COM  01002  84.0  01  01D  011     5    ABERGEMENT DE VAREY   
2     COM  01004  84.0  01  01D  011     1      AMBERIEU EN BUGEY   
3     COM  01005  84.0  01  01D  012     1    AMBERIEUX EN DOMBES   
4     COM  01006  84.0  01  01D  011     1                AMBLEON   

                  NCCENR                  LIBELLE   CAN  COMPARENT  
0  Abergement-Clémenciat  L'Abergement-Clémenciat  0108        NaN  
1    Abergement-de-Varey    L'Abergement-de-Varey  0101        NaN  
2      Ambérieu-en-Bugey        Ambérieu-en-Bugey  0101        NaN  
3    Ambérieux-en-Dombes      Ambérieux-en-Dombes  0122        NaN  
4                Ambléon                  Ambléon  0104        NaN  
1288
1288


#### b) Définition des fonctions permettant la connexion à l'API et la récupération des données selon les termes clés

In [11]:
# Définition de la fonction d'extraction de données

def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    try:
        df = pd.json_normalize(wb["results"]) # Conversion du fichier Json en dataframe pandas
    except json.JSONDecodeError:
        print("A file was empty")
    
    if 'longitude' in df.columns.tolist() and 'latitude' in df.columns.tolist():
    # Conversion du dataframe classique en dataframe géographique 
        dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["longitude"], df["latitude"]), crs = 4326) 
        dpe = dpe.dropna(subset = ['longitude', 'latitude'])

        return dpe

In [12]:
# Fixation des paramètres pour le requêtage
    #code_commune="01450"
code_commune = CODES[53]
size = 10000
root="https://data.ademe.fr/data-fair/api/v1/datasets/dpe-france/lines"
url_api = f"{root}?page=1&after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=max"
url_api2 = f"{root}?after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"


In [13]:
# Fonction qui retourne l'url à partir du code_commune
def ret_url (cod,size_):
    return f"{root}?after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{cod}" + "%22" + f"&size={size_}&select=" + "%2A&sampling=neighbors"


### c) Récupération des données relatives à l'ensemble des DPE réalisés en Ile-de-France et mises à disposition dans le portail Open data de l'Ademe

En requêtant l'API "DPE logements", on récupère tous les DPE disponibles pour chaque commune d'Ile-de-France puis on les unifie en un seul dataframe "dpe_f".

In [38]:
# Construction d'un dataframe complet des données pour la Région île-de-France

dpe_f = get_dpe_from_url(ret_url(CODES[0],10000))

for i in range(len(CODES)-1):                                                  # len(CODES)
    
    dpei = get_dpe_from_url(ret_url(CODES[i+1],10000))
    if dpei is not None:
        Commune = co_com2.loc[co_com2["COM"]==CODES[i],"NCC"].tolist()[0]
        dpei["Commune"] = Commune
        #dpei.set_index("Commune")
        dpe_f = pd.concat([dpe_f, dpei])
    


In [15]:
dpe_f.head()

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,version_methode_dpe,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,geometry,Commune
0,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,101,Logement,15 Rue de Tolbiac 75013 Paris,531934,...,V2012,3CL - DPE,75,434.00,2013-05-09,2.374836,None,T7b7O-zAyMijXFdlyWl7M,POINT (2.37484 48.82939),NaN
1,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,108,Logement,15 Rue de Tolbiac 75013 Paris,457143,...,V2012,3CL - DPE,75,434.00,2013-05-11,2.374836,None,99qI3IKDlsdqj_LgNBzmX,POINT (2.37484 48.82939),NaN
2,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,109,Logement,15 Rue de Tolbiac 75013 Paris,885078,...,V2012,3CL - DPE,75,434.00,2013-05-11,2.374836,None,RIrNiQLnIG9Iqsg19e8aC,POINT (2.37484 48.82939),NaN
3,D,Vente,1940,"48.847662,2.28565",48.847662,29.00,119,Logement,11 Rue Beaugrenelle 75015 Paris,700764,...,V2012,3CL - DPE,75,170.02,2013-05-13,2.285650,None,XZIHgdlUOrEuAIrcpa9xa,POINT (2.28565 48.84766),NaN
4,N,Vente,1900,"48.839935,2.332912",48.839935,6.99,123,Logement,11 Rue Campagne Première 75014 Paris,215124,...,V2012,3CL - DPE,75,0.00,2013-05-13,2.332912,None,uVKCNovGIFXlkwZwxcXWD,POINT (2.33291 48.83993),NaN


In [16]:
dpe_f.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1323802 entries, 0 to 23
Data columns (total 24 columns):
 #   Column                           Non-Null Count    Dtype   
---  ------                           --------------    -----   
 0   classe_consommation_energie      1323802 non-null  object  
 1   tr001_modele_dpe_type_libelle    1323802 non-null  object  
 2   annee_construction               1323802 non-null  int64   
 3   _geopoint                        1323802 non-null  object  
 4   latitude                         1323802 non-null  float64 
 5   surface_thermique_lot            1323726 non-null  float64 
 6   _i                               1323802 non-null  int64   
 7   tr002_type_batiment_description  1323802 non-null  object  
 8   geo_adresse                      1323802 non-null  object  
 9   _rand                            1323802 non-null  int64   
 10  code_insee_commune_actualise     1323802 non-null  object  
 11  estimation_ges                   1

### 2) Récupération du second jeu de données via le package Pynsee: données socio-démographiques sur les communes (source: dispositif Filosofi, Insee)

#### a) Installation du package pynsee disponible seulement sous Github

In [18]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.5 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.0 MB/s eta 0:00:00
  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-humtm93t
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-humtm93t
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit 2d11fcab74f74f978cd4aa965a922153c660e5f6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 9.8 MB/s eta 0:00:00
  Created wheel for pynsee: filename=pynsee-0.1.2-py3-none-any.whl size=9804928 sha256=a0bcdb7214ab8ff69ec2ebe09add245174ebc48d02e806deb3a46c373a159930
  Stored in directory: /tmp/pip-ephem-wheel-cache-pv

In [19]:
# Importation du package:
import pynsee
import pynsee.download

#### b) Récupération du fichier Filosofi 2016 au niveau communal

In [32]:
Filosofi = pynsee.download.download_file("FILOSOFI_COM_2016")

Previously saved data has been used:
/home/onyxia/.cache/pynsee/pynsee/78281770afe21f56f6df245bd6f30ea2
Set update=True to get the most up-to-date data


Extracting: 100%|██████████| 10.6M/10.6M [00:00<00:00, 150MB/s]


In [33]:
Filosofi.head()

,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,PIMP16,TP6016,TP60AGE116,TP60AGE216,TP60AGE316,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
0,01001,L'Abergement-Clémenciat,313,795.5,22679,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,101,248,24382.083333333336,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01004,Ambérieu-en-Bugey,6363,14228,19721,49,17,19,22,22,...,27,8.8,6.9,2.8,2.1,2,-15.7,10457.083333333334,33880.555555555555,3.239962279688143
3,01005,Ambérieux-en-Dombes,633,1662.5,23378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01006,Ambléon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
Filosofi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34932 entries, 0 to 34931
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODGEO           34932 non-null  object
 1   LIBGEO           34932 non-null  object
 2   NBMENFISC16      31405 non-null  object
 3   NBPERSMENFISC16  31405 non-null  object
 4   MED16            31405 non-null  object
 5   PIMP16           5293 non-null   object
 6   TP6016           4372 non-null   object
 7   TP60AGE116       759 non-null    object
 8   TP60AGE216       1414 non-null   object
 9   TP60AGE316       1826 non-null   object
 10  TP60AGE416       1292 non-null   object
 11  TP60AGE516       890 non-null    object
 12  TP60AGE616       403 non-null    object
 13  TP60TOL116       2155 non-null   object
 14  TP60TOL216       3174 non-null   object
 15  PACT16           5293 non-null   object
 16  PTSA16           5293 non-null   object
 17  PCHO16           5293 non-null 

## B) Retraitement des données

### 1) Sélection des variables pertinentes pour l'étude

#### a) Dans le dataframe dpe_f

In [26]:
dpe_f=dpe_f.loc[:,["_id","date_etablissement_dpe","consommation_energie","classe_consommation_energie","estimation_ges","classe_estimation_ges","annee_construction","surface_thermique_lot","Commune","code_insee_commune_actualise","longitude","latitude","tr002_type_batiment_description","tv016_departement_code","_geopoint","geometry"]]
dpe_f.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1323802 entries, 0 to 23
Data columns (total 16 columns):
 #   Column                           Non-Null Count    Dtype   
---  ------                           --------------    -----   
 0   _id                              1323802 non-null  object  
 1   date_etablissement_dpe           1323802 non-null  object  
 2   consommation_energie             1323802 non-null  float64 
 3   classe_consommation_energie      1323802 non-null  object  
 4   estimation_ges                   1323802 non-null  float64 
 5   classe_estimation_ges            1323802 non-null  object  
 6   annee_construction               1323802 non-null  int64   
 7   surface_thermique_lot            1323726 non-null  float64 
 8   Commune                          1313850 non-null  object  
 9   code_insee_commune_actualise     1323802 non-null  object  
 10  longitude                        1323802 non-null  float64 
 11  latitude                         1

#### b) Dans le dataframe Filosofi

In [35]:
Filosofi=Filosofi.loc[:,["CODGEO","LIBGEO","NBMENFISC16","NBPERSMENFISC16","MED16"]]
Filosofi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34932 entries, 0 to 34931
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODGEO           34932 non-null  object
 1   LIBGEO           34932 non-null  object
 2   NBMENFISC16      31405 non-null  object
 3   NBPERSMENFISC16  31405 non-null  object
 4   MED16            31405 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


### 2) Nettoyage et mise en forme de certaines variables

#### a) Cleaning de la variable "code_commune_actualise" dans le df "dpe_f"

En examinant les données, il est apparu que la variable "code_commune_actualise" comportait un certain nombre de modalités mal renseignées: des codes commune de moins de 5 chiffres,des codes communes comportant le signe ".", des codes commune comportant des noms de commune, etc. Aussi un nettoyage de cette variable clé s'impose: 

In [39]:

# liste des codes commune exisistant initialement dans le fichier DPE brut:
a=dpe_f["code_insee_commune_actualise"].unique().tolist()

# on transforme les éléments de la liste en str:
b=[str(x) for x in a]

# on code le filtre de "nettoyage" à appliquer à notre df "dpe_f":
filtre = [x for x in b if len(x) == 5 if "." not in x if x[:2] in (["75","77","91","92","93","94","95"])]
# on ne garde que les code commune composés de 5 chiffres, sans "." et débutant par les nombres suivants:
# 75, 77, 91, 92, 93, 94 et 95.

print(len(filtre)) # nombre de codes commune retenus in fine pour l'IDF.

# Application du filtre au df "dpe_f":
dpe_f2=dpe_f[dpe_f["code_insee_commune_actualise"].isin(filtre)].copy()
# tous les codes commune sont clean dans "dpe_f2".
dpe_f2.shape

1027


(1134380, 24)

In [40]:
dpe_f.shape # on a donc supprimé 189 422 lignes du df "dpe_f" suite au nettoyage des codes commune.

(1323802, 24)

#### b) Dans le df "Filosofi", passage au format numérique des variables "NBMENFISC16", "NBPERSMENFISC16" et "MED16"  

In [ ]:
# Modification des types des variables: passage au format numérique de toutes les variables sauf codgeo et libgeo.
liste_var=Filosofi.columns.tolist()[2:]
for var in liste_var:
    Filosofi[var]=pd.to_numeric(Filosofi[var])
Filosofi.info()

### 3) Création de nouvelles variables utiles pour l'analyse statistique des données

#### a) Dans le df dpe_f2:

In [42]:
# Calcul de nouvelles variables dans dpe_f: année du DPE, âge du logement, département.

# 1) Création d'une variable "année" de réalisation du DPE:
dpe_f2["annee"]=dpe_f2.date_etablissement_dpe.str[:4]

# 2) Création d'une variable "âge" du logement:
# il faut d'abord transformer la variable "année" en int64:
dpe_f2["annee2"]=pd.to_numeric(dpe_f2["annee"])
dpe_f2["age_logement"]=dpe_f2["annee2"]-dpe_f2["annee_construction"]

# 3) Création d'une variable "département" à partir du code de la commune du DPE:
dpe_f2["dep"]=dpe_f2.code_insee_commune_actualise.str[:2]

# Discrétisation de variables quantitatives:

#dpe_f2['classe_age'] = pd.cut(dpe_f2['age_logement'],bins=[5,10,25,50,100],\
#                                      labels=["Part nulle d'énergivores","Part énergivores<20%",\
#                                              "Part énergivores entre 20% et 50%",\
#                                              "Part énergivores>50%"],\
#                                     ordered=False)
# TODO: classes de niveaux de conso d'énergie; classes de surface thermique...

In [43]:
dpe_f2.head()

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,date_etablissement_dpe,longitude,_score,_id,geometry,Commune,annee,annee2,age_logement,dep
0,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,101,Logement,15 Rue de Tolbiac 75013 Paris,531934,...,2013-05-09,2.374836,None,T7b7O-zAyMijXFdlyWl7M,POINT (2.37484 48.82939),NaN,2013,2013,43,75
1,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,108,Logement,15 Rue de Tolbiac 75013 Paris,457143,...,2013-05-11,2.374836,None,99qI3IKDlsdqj_LgNBzmX,POINT (2.37484 48.82939),NaN,2013,2013,43,75
2,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,109,Logement,15 Rue de Tolbiac 75013 Paris,885078,...,2013-05-11,2.374836,None,RIrNiQLnIG9Iqsg19e8aC,POINT (2.37484 48.82939),NaN,2013,2013,43,75
3,D,Vente,1940,"48.847662,2.28565",48.847662,29.00,119,Logement,11 Rue Beaugrenelle 75015 Paris,700764,...,2013-05-13,2.285650,None,XZIHgdlUOrEuAIrcpa9xa,POINT (2.28565 48.84766),NaN,2013,2013,73,75
4,N,Vente,1900,"48.839935,2.332912",48.839935,6.99,123,Logement,11 Rue Campagne Première 75014 Paris,215124,...,2013-05-13,2.332912,None,uVKCNovGIFXlkwZwxcXWD,POINT (2.33291 48.83993),NaN,2013,2013,113,75


#### b) Dans le df Filosofi

In [ ]:
# Discrétisation de la variable "MED15": niveau de vie médian de la commune
# TODO

### 4) Agrégation du df "dpe_f2" par commune: création du df "df_dpe_com" 

In [52]:
# On commence par transformer les variables catégorielles en indicatrices:
# Variable 1: classe de consommation d'énergie
classe_conso=pd.get_dummies(dpe_f2["classe_consommation_energie"])
classe_conso.columns=["classe_conso_A","classe_conso_B","classe_conso_C","classe_conso_D","classe_conso_E",
                     "classe_conso_F","classe_conso_G","classe_conso_H","classe_conso_N"]
classe_conso.head()

# Variable 2: classe d'émission de gaz à effet de serre (classe_estimation_ges)
classe_ges=pd.get_dummies(dpe_f2["classe_estimation_ges"])
classe_ges.columns=["classe_ges_A","classe_ges_B","classe_ges_C","classe_ges_D","classe_ges_E",
                     "classe_ges_F","classe_ges_G","classe_ges_H","classe_ges_N"]
classe_ges.head()

# Variable 3: type de logement (tr002_type_batiment_description):
type_logements=pd.get_dummies(dpe_f2["tr002_type_batiment_description"])
type_logements.columns=["type_bat_coll","type_logement","type_maison"]
type_logements.head()

,type_bat_coll,type_logement,type_maison
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [56]:
Codes_com_dpe_f2 =dpe_f2[["code_insee_commune_actualise"]]
x = Codes_com_dpe_f2.merge(classe_conso, how = "left", left_index = True, right_index = True)
classe_conso_com=x.groupby('code_insee_commune_actualise').mean().reset_index()
classe_conso_com

,code_insee_commune_actualise,classe_conso_A,classe_conso_B,classe_conso_C,classe_conso_D,classe_conso_E,classe_conso_F,classe_conso_G,classe_conso_H,classe_conso_N
0,75056,0.046905,0.192078,0.102842,0.231791,0.191372,0.071158,0.026051,8.858408e-07,0.137803
1,75101,0.046438,0.189422,0.092931,0.237074,0.208804,0.080566,0.029799,1.686989e-06,0.114965
2,75102,0.047018,0.187907,0.090306,0.237980,0.211544,0.081856,0.030451,1.852692e-06,0.112936
3,75103,0.045053,0.192730,0.095626,0.235210,0.200561,0.075868,0.027799,1.199740e-06,0.127152
4,75104,0.046377,0.188954,0.094272,0.237077,0.205644,0.078834,0.029030,1.458623e-06,0.119810
...,...,...,...,...,...,...,...,...,...,...
1022,95676,0.049310,0.077621,0.072673,0.242483,0.259854,0.124072,0.052828,0.000000e+00,0.121159
1023,95678,0.051460,0.084796,0.072140,0.241027,0.255269,0.123644,0.051945,0.000000e+00,0.119720
1024,95680,0.042579,0.196872,0.102308,0.232035,0.191527,0.070638,0.025450,1.572809e-06,0.138590
1025,95682,0.044951,0.062822,0.070826,0.251069,0.269378,0.128275,0.055915,0.000000e+00,0.116764


In [50]:
dpe_f2["tr002_type_batiment_description"].value_counts()

,Bâtiment collectif à usage principal d'habitation,Logement,Maison Individuelle
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


### 5) Jointure entre les dataframes "Filisofi" et "df_dpe_com": création du df "df_dpe_filo_com"

In [ ]:
### Sauvegarde des dataframes finaux sous forme de fichiers .csv dans le repo de Github

In [ ]:
# Exporte le dataframe dpe_f2 dans le csv: 
#dpe_f2.to_csv("'~/work/projetpy/data_science_project/dpe_f2.csv", index=False)